<a href="https://colab.research.google.com/github/olujozef/ARF-ARENA/blob/main/Office_hours_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SETUP

In [85]:
import einops
import torch as t
from torch import Tensor
from typing import Tuple

def assert_all_equal(actual: Tensor, expected: Tensor) -> None:
    assert actual.shape == expected.shape, f"Shape mismatch, got: {actual.shape}"
    assert (actual == expected).all(), f"Value mismatch, got: {actual}"
    print("Tests passed!")


def assert_all_close(actual: Tensor, expected: Tensor, atol=1e-3) -> None:
    assert actual.shape == expected.shape, f"Shape mismatch, got: {actual.shape}"
    t.testing.assert_close(actual, expected, atol=atol, rtol=0.0)
    print("Tests passed!")

Task 1

In [86]:
def copy_along_dimension(input: Tensor, copies: int) -> Tensor:
    """Create a new tensor by copying the input tensor along a new dimension at the end.

    input: Input tensor of any shape
    copies: Number of copies to make along the new dimension

    Return: Tensor with shape (*tens.shape, copies) where the last dimension contains
    identical copies of the original tensor.

    Use einops operations to implement this function.
    """

    from einops import repeat
    return repeat(input, "... -> ... copies", copies=copies)
input = t.tensor([1, 2, 3])
expected_result = t.tensor([[1, 1],
                            [2, 2],
                            [3, 3]])
assert_all_equal(copy_along_dimension(input, 2), expected_result)

Tests passed!


In [48]:
from einops import repeat, rearrange
def copy_along_dimension(input: Tensor, copies: int) -> Tensor:

    return rearrange(repeat (input, 'h w -> (h w) copies', copies =copies), '(h w) copies -> (h w) copies', h=input.shape[0], w=input.shape[1])
input = t.tensor([[1, 2], [3, 4]])  # shape: (2, 2)
expected_result = t.tensor([[1, 1, 1],
                            [2, 2, 2],
                            [3, 3, 3],
                            [4, 4, 4]])
assert_all_equal(copy_along_dimension(input, 3), expected_result)  # → (2, 2) from (4, 3)
print(expected_result.shape)


Tests passed!
torch.Size([4, 3])


Task 2

In [51]:
def compute_ray_lengths(origins: Tensor, directions: Tensor,
                       t_values: Tensor) -> Tensor:
    """
    Calculate the 3D positions along rays at different distances.
    Then compute the length of each resulting position vector (distance from origin).

    Parameters:
    - origins: The origin points of rays
    - directions: The direction vectors of rays
    - t_values: Distance values along each ray

    Returns:
    - lengths: The distance of each point from (0,0,0)

    Hint: For each ray, the point at distance t is: origin + t * direction
          The length of a position vector is its Euclidean distance from (0,0,0)

    What to do:
    #find the lengths = t.linalg.norm(points, dim=-1)
    points = origin + t_values * direction
    # origin = (3,3)
    # direction = (3,3)
    # t_values = (3). We should reshape it to (3,1) for easy elementwise multiplication with the direction

    """
    ### To Reshape t_value
    reshaped_t_values = t_values.reshape(-1, 1)  # Now t_value is same shape with the others.

    ### multiply t_values by the direction
    broadcasted  = reshaped_t_values * directions

    ### Point at distance t
    points = origins + broadcasted

    ### Now to find the lengths from (0,0,0), i.e the Euclidean distance
    lengths = t.linalg.norm(points, dim=-1)
    return lengths

origins = t.tensor([
    [0.0, 0.0, 0.0],
    [1.0, 0.0, 0.0],
    [0.0, 2.0, 0.0]
])

directions = t.tensor([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0]
])

t_values = t.tensor([5.0, 4.0, 3.0])
lengths = compute_ray_lengths(origins, directions, t_values)
expected = t.tensor([5.0, t.sqrt(t.tensor(17.0)), t.sqrt(t.tensor(13.0))])

assert_all_close(compute_ray_lengths(origins, directions, t_values),
                     expected)


Tests passed!


In [64]:
def compute_final_positions_and_distances(positions: Tensor, velocities: Tensor, times: Tensor) -> Tuple  [Tensor, Tensor]:

    """
    Problem: Compute Final Positions of Moving Objects in 3D Space
    Imagine you have a set of objects in 3D space. Each one has:

    an initial position

    a velocity vector (how fast and in what direction it’s moving)

    and a time duration it's been moving.

    You need to calculate:

    The final position of each object.

    The distance from the origin (0,0,0) after moving.

    📌 Task:
    Implement the function:



        Compute the final positions of moving objects and their distances from the origin.

        Parameters:
        - positions: (N, 3) tensor of initial positions
        - velocities: (N, 3) tensor of velocity vectors
        - times: (N,) tensor of time durations

        Returns:
        - final_positions: (N, 3) tensor of new positions
        - distances: (N,) tensor of distances from the origin

    Final position = initial_position + velocity * time

    Distance from origin = Euclidean norm = torch.linalg.norm(position, dim=-1)

    """
# Reshape time
    reshaped_times = times.reshape(-1, 1)

    broadcasted = reshaped_times * velocities
# final postion
    final_positions = positions + broadcasted

# Distance
    distances = t.linalg.norm(final_positions, dim=-1)
    return final_positions, distances

positions = t.tensor([
    [0.0, 0.0, 0.0],
    [1.0, 2.0, 3.0],
    [4.0, 0.0, 0.0]
])

velocities = t.tensor([
    [1.0, 0.0, 0.0],
    [0.0, 2.0, 0.0],
    [0.0, 0.0, 1.0]
])

times = t.tensor([2.0, 3.0, 4.0])

final_positions, distances = compute_final_positions_and_distances(positions, velocities, times)

print("Final Positions:\n", final_positions)
print("Distances from Origin:\n", distances)

Final Positions:
 tensor([[2., 0., 0.],
        [1., 8., 3.],
        [4., 0., 4.]])
Distances from Origin:
 tensor([2.0000, 8.6023, 5.6569])


Task 3(Bonus)

In [92]:
def reverse_tensor(input: Tensor) -> Tensor:
    """Reverse the order of elements in the input tensor.

    input: 1D tensor

    Return: Tensor with the same shape as x but with elements in reverse order.
    Use einops.rearrange to implement this without using torch.flip.
    """
    from einops import rearrange
    length = input.shape[0]
    reversed_indices = t.arange(length - 1, -1, -1)
    reversed_input = input[reversed_indices]
    return rearrange(reversed_input, 't -> t')


input = t.tensor([1, 2, 3, 4, 5])
expected = t.tensor([5, 4, 3, 2, 1])
assert_all_equal(reverse_tensor(input), expected)

Tests passed!
